In [68]:
import pandas as pd
import numpy as np
import sklearn
import glob
from pathlib import Path 
import sys
sys.path.append('../mss')
import mssmain as mss
import peakutils
from scipy.integrate import simps
from ast import literal_eval
import scipy
from tqdm import tqdm

In [2]:
#Read the dataset
path = '../example_data/peakdata/labelled_output/'
all_files = glob.glob(path + "/*.csv")

In [3]:
for i in range(len(all_files)):
    if i == 0:
        df = pd.read_csv(all_files[i])
        df['source'] = all_files[i]
    else:
        df_else = pd.read_csv(all_files[i])
        df_else['source'] = all_files[i]
        df = df.append(df_else, ignore_index = True)

In [4]:
#reshape data
df.columns = ['index', 'mz', 'i array', 'label', 'source']
df = pd.DataFrame(df, columns = ['mz', 'i array', 'label', 'source', 'index'])
df.head()

,mz,i array,label,source,index
0,593.411581,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,../example_data/peakdata/labelled_output\100ba...,680
1,196.997350,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,../example_data/peakdata/labelled_output\100ba...,60
2,327.006530,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,../example_data/peakdata/labelled_output\100ba...,280
3,420.975690,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,../example_data/peakdata/labelled_output\100ba...,488
4,483.345794,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,../example_data/peakdata/labelled_output\100ba...,568


# RT conversion rate should be incorporated if any width parameters included

In [5]:
df_relabel = df[(df['label'] != 3) & (df['label'] != 2) & (df['label'] != 1)] #df for mislabelled peaks

In [6]:
df_model = df.drop(df_relabel.index) #data for modeling, now have ~3500 rows of data

In [26]:
rt_conversion_rate = 0.005533333

In [88]:
def peak_para(intensity, rt_conversion_rate, peak_thres = 0.01, thr = 0.02, min_d = 1, rt_window = 1.5, peak_area_thres = 1e5, min_scan = 15, max_scan = 200, max_peak = 5, min_scan_window = 20, sn_range = 7):
    '''
    firstly get rt, intensity from given mz and error out of the mzml file
    Then find peak on the intensity array, represent as index --> index
    Find peak range by looping from peak index forward/backward until hit the peak_base --> l_range,h_range. peakspan = h_range - l_range
    Trim/correct peak range is too small or too large, using min_scan/max_scan,min_scan_window --> trimed l/h_range
    Integration of peak based on the given range using simp function --> peakarea
    '''
    
    #Get rt_window corresponded scan number -- needs update later
    
    #Get peak index
    indexes = peakutils.indexes(intensity, thres=thr, min_dist = min_d)
    
    result_dict = {}
    
    
    #dev note: boundary detection refinement
    for index in indexes:
        h_range = index
        l_range = index
        base_intensity = peak_thres * intensity[index] # use relative thres, also considering S/N, 1/2 rt point?
        half_intensity = 0.5 * intensity[index]

        #Get the higher and lower boundary
        while intensity[h_range] >= base_intensity:
            h_range += 1
            if intensity[h_range-1] < half_intensity: #potentially record this
                if h_range - index > 4: #fit r2 score, keep record https://stackoverflow.com/questions/55649356/how-can-i-detect-if-trend-is-increasing-or-decreasing-in-time-series as alternative
                    x = np.linspace(h_range - 2, h_range, 3)
                    y = intensity[h_range - 2 : h_range + 1]
                    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
#                     print(rt[h_range],r_value)
                    if abs(r_value) < 0.6:
                        break
                    elif h_range > len(intensity)-2: 
                        break
        while intensity[l_range] >= base_intensity: #Dev part 2, low priority since general peak shapes
            l_range -= 1
            if intensity[l_range] < half_intensity:
                pass #backdoor for recording 1/2 rt point
        #Output a range from the peak list
    
        peak_range = intensity[l_range:h_range]#no filter so ignored for tailing effects
        #print(index + scan_window)
                
        #Calculate for S/N
        signal = intensity[index]
        neighbour_blank = intensity[l_range - sn_range : l_range] + intensity[h_range + 1 : h_range + sn_range + 1]
        noise = max(neighbour_blank)
        if noise != 0:
            sn = round(signal/noise, 3)
        else:
            sn = 0
        
        #Calculate height/width, consider log10 transform
        height = signal
        width = (h_range - l_range) * rt_conversion_rate
        #Add rt conversion factor here to convert width in scan into rt
        hw_ratio = round(height/width,0)
        

        #Intergration based on the simps function
        if len(peak_range) >= min_scan:
            integration_result = simps(peak_range)
            if integration_result >= peak_area_thres:
                #Calculate Area/background ratio, i.e, peak area vs rectangular area as whole(if =1 then peak is a pleateu)
                background_area = (h_range - l_range) * height
                ab_ratio = round(integration_result/background_area, 3)
                
                #appending to result
                if len(result_dict) == 0:
                    result_dict.update({index : [l_range, h_range, integration_result, sn, hw_ratio, ab_ratio]})
                elif integration_result != list(result_dict.values())[-1][2]: #Compare with previous item
                    s_window = abs(index - list(result_dict.keys())[-1])
                    if s_window > min_scan_window:
                        result_dict.update({index : [l_range, h_range, integration_result, sn, hw_ratio, ab_ratio]})

                    
        #Filtering:
        #1. delete results that l_range/h_range within 5 scans
        #3. If still >5 then select top 5 results
        #list(result_dict.values())[-1]
    
    #Noise filter
    if len(result_dict) > max_peak:
        result_dict = {}
        


    return result_dict

In [72]:
test_array = literal_eval(df_model.iloc[1]['i array'])

In [73]:
para = peak_para(test_array, rt_conversion_rate)

In [97]:
df_para = pd.DataFrame(columns = ['mz', 'assymetric', 'integration', 'sn', 'hw', 'ab','label'])

In [98]:
for i, row in tqdm(df_model.iterrows()):
    try:
        i_array = literal_eval(row['i array'])
        para = peak_para(i_array, rt_conversion_rate)

        for i in para.items():
            index = i[0]
            l_range = i[1][0]
            h_range = i[1][1]
            integration = i[1][2]
            sn = i[1][3]
            hw = i[1][4]
            ab = i[1][5]

            paradict = {'mz' : row['mz'],
                        'assymetric' : [(l_range - index) * rt_conversion_rate, (h_range - index) * rt_conversion_rate],
                       'integration' : integration,
                       'sn' : sn,
                       'hw' : hw,
                       'ab' : ab,
                       'label': row['label']}
            df_para = df_para.append(paradict, ignore_index = True)
    except:
        continue

3555it [02:30, 23.68it/s]


In [99]:
df_para

,mz,assymetric,integration,sn,hw,ab,label
0,196.997350,"[-0.005533333, 0.199199988]",2.961758e+05,0.892,36466.0,1.072,3
1,196.997350,"[-0.011066666, 0.105133327]",1.408043e+05,0.915,67609.0,0.853,3
2,420.975690,"[-0.049799997, 0.13833332499999998]",6.973428e+05,0.000,195821.0,0.557,1
3,483.345794,"[-0.022133332, 0.12173332599999999]",1.797947e+05,0.000,59442.0,0.809,1
4,701.404588,"[-0.011066666, 0.171533323]",2.018438e+05,1.212,39443.0,0.849,1
...,...,...,...,...,...,...,...
4498,205.086569,"[-0.33199998, 0.11066666]",1.719529e+06,2.071,42805.0,1.134,1
4499,205.086569,"[-0.45373330599999995, 0.188133322]",1.965307e+06,1.674,14256.0,1.852,1
4500,568.551435,"[-0.011066666, 0.099599994]",2.873902e+05,1.067,206904.0,0.628,2
4501,568.551435,"[-0.16046665699999998, 0.011066666]",3.588770e+05,0.248,30983.0,2.178,2


In [100]:
df_para.to_csv('../example_data/peakdata/labelled_output/summary-1stedit.csv')